In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [17]:
zip_code = '07920'
source = requests.get(f'https://www.cellreception.com/search.php?zip={zip_code}&page=1').text
soup = BeautifulSoup(source, 'lxml')

print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html>
 <head>
  <title>
   Basking Ridge, NJ 07920 Carrier Reviews &amp; Coverage Maps | CellReception
  </title>
  <meta content="Who's the Best Carrier In Basking Ridge? 89 People Agree It's This." property="og:title"/>
  <meta content="http://www.cellreception.com/search.php?zip=07920&amp;page=1" property="og:url"/>
  <meta content="http://images.cellreception.com/inc/screenshot.jpg" property="og:image"/>
  <meta content="CellReception" property="og:site_name"/>
  <meta content="The best signal in the neighborhood is..." property="og:description"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Coverage reviews of Basking Ridge, NJ 07920. Service was good a few years ago but canât get a call in my house and calls drop..." name="description"/>
  <link href="https://www.cellreception.com/search.php?zip=07920&amp;page=1" r

In [68]:
# zip_code = '07027'
# source = requests.get(f'https://www.cellreception.com/search.php?zip=07920&page=1000')
# print(source.status_code)
# html = source.text
# soup = BeautifulSoup(html, 'lxml')
# print(soup.prettify())

here


In [72]:
reviews = soup.findAll(
    lambda tag: tag.name=='div' and 
    tag.has_attr('itemtype') and 
    tag['itemtype']=="http://data-vocabulary.org/Review"
)

test = reviews[0]
carrier = test.find('h2').text
review = test.find('em').text
actual_review = test.find('p').text
print(repr(actual_review))

# rows = table.findAll(lambda tag: tag.name=='tr')
# parsed_rows = [row.text.replace("\n",'') for row in rows]
# parsed_rows

'\n\nService was good a few years ago but canâ\x80\x99t get a call in my house and calls drop constantly\n\n'


In [64]:
import re
text = text.replace("\n",'')
pattern = '\((\d) stars?\).*(\d{5}) \|(.*)'

m = re.search(pattern, text)
print(m)
if m is not None:
    _stars = m.group(1)
    _review_zip_code = m.group(2)
    _date = m.group(3)
    
    print(_stars, _review_zip_code, _date)

<re.Match object; span=(0, 78), match='(1 star) |Goldfinch Dr. | Meadow Rd.,Bridgewater,>
1 08807 September 06, 2020


In [173]:
def scrape_all_zips_reviews(zip_list):
    _pattern = '\((\d) stars?\).*(\d{5}) \|(.*)'
    _columns = ['zip_code', 'date', 'carrier', 'rating', 'review', 'review_zip_code']
    
    all_df = pd.DataFrame(columns=_columns)
    for _zc in zip_list:
        _zip_df = pd.DataFrame(columns=_columns)
        for i in range(1,6):
            source = requests.get(f'https://www.cellreception.com/search.php?zip={_zc}&page={i}')
            if source.status_code == 404:
                break
                
            html = source.text
            soup = BeautifulSoup(html, 'lxml')        
            
            review_blocks = soup.findAll(
                lambda tag: tag.name=='div' and 
                tag.has_attr('itemtype') and 
                tag['itemtype']=="http://data-vocabulary.org/Review"
            )
       
            if review_blocks == None:
                continue
            
            for _rb in review_blocks:
                _carrier = _rb.find('h2').text
                _info = _rb.find('em').text
                _review = _rb.find('p').text
                
                if _info == None:
                    continue
                _info = _info.replace("\n",'')
                
                m = re.search(_pattern, _info)
                if m is not None:
                    
                    _stars = m.group(1)
                    _review_zip_code = m.group(2)
                    _date = m.group(3)
                    
                    _zip_df = _zip_df.append({
                        'zip_code': _zc,
                        'date': _date,
                        'carrier': _carrier,
                        'rating': _stars,
                        'review': _review,
                        'review_zip_code': _review_zip_code,
                    }, ignore_index=True)
        
        all_df = pd.concat([all_df, _zip_df])
        
    return all_df

In [174]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def scrape_in_equal_chunks(num_parts):
    _chunks = list(chunks(to_scrape_list, int(len(to_scrape_list)/num_parts)+1))

to_scrape = pd.read_csv('zips_fios_project_to_scrape.csv', dtype={'zip_code': str})
to_scrape_list = list(to_scrape['zip_code'])
scraped_reviews = scrape_all_zips_reviews(to_scrape_list)
scraped_reviews.head()

,zip_code,date,carrier,rating,review,review_zip_code
0,01420,"September 29, 2020",T-Mobile,0,\n\nT Mobile sucks hard! No coverage even when...,01420
1,01420,"August 27, 2020",Verizon,1,"\n\nTerrible service. Dropped calls,can't gets...",01469
2,01420,"March 05, 2020",US Cellular,4,\n\nWe have service everywhere but it's extrem...,03071
3,01420,"February 03, 2020",MetroPCS,1,\n\nI hardly ever have signal. My phone doesn'...,01469
4,01420,"January 06, 2020",AT&T,2,\n\nFrequently have only two bars in my reside...,01453


In [175]:
scraped_reviews.to_csv('recent_carrier_reviews_by_zip_code.csv', index=False)

## Scrape overall carrier reviews per zip

In [84]:
to_scrape = pd.read_csv('sample_zips_to_scrape.csv', dtype={'zip': str})
to_scrape_list = list(to_scrape['zip'])

def scrape_all_zips():
    pattern = '(Verizon|AT&T|Sprint|T-Mobile) \((\d*\.?\d*) stars \| (\d*) Reviews\)'
    all_df = pd.DataFrame(columns=['zip_code', 'carrier', 'num_stars', 'num_reviews'])
    for _zc in to_scrape_list:
        source = requests.get(f'https://www.cellreception.com/search.php?zip={_zc}&page=1').text
        soup = BeautifulSoup(source, 'lxml')
        
        table = soup.find(
            lambda tag: tag.name=='div' and 
            tag.has_attr('itemtype') and 
            tag['itemtype']=="http://data-vocabulary.org/Review-aggregate"
        )

        if table == None:
            continue
        
        rows = table.findAll(lambda tag: tag.name=='tr')
        
        if rows == None:
            continue
        
        parsed_rows = [row.text.replace("\n",'') for row in rows]

        zip_df = pd.DataFrame(columns=['zip_code', 'carrier', 'num_stars', 'num_reviews'])
        for r in parsed_rows[1:]:
            m = re.search(pattern, r)
            if m is not None:
                _carrier = m.group(1)
                _stars = m.group(2)
                _reviews = m.group(3)
                zip_df = zip_df.append({
                    'zip_code': _zc,
                    'carrier': _carrier, 
                    'num_stars': _stars, 
                    'num_reviews': _reviews
                } , ignore_index=True)
        all_df = pd.concat([all_df, zip_df])
        
    return all_df

In [85]:
results = scrape_all_zips()

In [86]:
results.to_csv('cellular_scraped_LW.csv', index=False)